In [1]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

from ocean_forest.random_forest import load, clean_data, dump_model, load_model
import xg_boost



In [2]:
def objective(trial, env="pp-mattei"):
    df = load(env=env)
    X,y = clean_data(df, env=env, depths=True)
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    param = {
        "nthread" : -1,
        'learning_rate': 0.8,
        #"silent": 1,
        #"tree_method": trial.suggest_categorical("tree_method", ["approx", "hist"]),
        'tree_method': trial.suggest_categorical('tree_method', ['approx', 'hist']),
        #"tree_method": "hist",
        "min_child_weight" : trial.suggest_int("min_child_weight", 1, 250),
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 25, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 12)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    bst = xgb.train(param, dtrain, evals=[(dtest, "validation")])#, callbacks=[pruning_callback])
    preds = bst.predict(dtest)
    accuracy = sklearn.metrics.r2_score(test_y, preds)
    return accuracy



In [3]:
def objective2(trial, env="pp-mattei"):
    df = load(env=env)
    X,y = clean_data(df, env=env, depths=True)
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    param = {
        #'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        #'sampling_method': 'gradient_based',
        'lambda': trial.suggest_float('lambda', 7.0, 17.0, log=True),
        'alpha': trial.suggest_float('alpha', 7.0, 17.0, log=True),
        'eta': trial.suggest_categorical('eta', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'gamma': trial.suggest_categorical('gamma', [18, 19, 20, 21, 22, 23, 24, 25]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'colsample_bynode': trial.suggest_categorical('colsample_bynode', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 8, 600),  
        'max_depth': trial.suggest_categorical('max_depth', [3, 4, 5, 6, 7]),  
        'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
        'random_state': 42
    }

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    bst = xgb.train(param, dtrain, evals=[(dtest, "validation")]) #, callbacks=[pruning_callback])
    preds = bst.predict(dtest)
    accuracy = sklearn.metrics.r2_score(test_y, preds)
    return accuracy


In [6]:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=1000, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


[I 2024-12-17 20:30:45,845] A new study created in memory with name: no-name-11ec287d-5991-4162-a68d-be5b9be4825f


Use depth as feature
[0]	validation-rmse:1.23057
[1]	validation-rmse:1.17470
[2]	validation-rmse:1.13557
[3]	validation-rmse:1.12291
[4]	validation-rmse:1.09210
[5]	validation-rmse:1.08564
[6]	validation-rmse:1.07596
[7]	validation-rmse:1.07153
[8]	validation-rmse:1.05066
[9]	validation-rmse:1.03898


[I 2024-12-17 20:30:45,913] Trial 0 finished with value: 0.46700978822454875 and parameters: {'tree_method': 'hist', 'min_child_weight': 126, 'booster': 'dart', 'lambda': 4.7381160592718024e-05, 'alpha': 2.937491891562188e-08, 'subsample': 0.9522497002484465, 'colsample_bynode': 0.7265475858542689, 'reg_lambda': 0.15599651761131902, 'max_depth': 2, 'eta': 2.0093953087626883e-07, 'gamma': 0.0968485403755674, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 5.850674798797878e-06, 'skip_drop': 0.0007690676648318273}. Best is trial 0 with value: 0.46700978822454875.


Use depth as feature
[0]	validation-rmse:1.08667
[1]	validation-rmse:1.01362
[2]	validation-rmse:0.98177
[3]	validation-rmse:0.98105
[4]	validation-rmse:0.97108
[5]	validation-rmse:0.95190
[6]	validation-rmse:0.93887
[7]	validation-rmse:0.92023
[8]	validation-rmse:0.91656
[9]	validation-rmse:0.91064


[I 2024-12-17 20:30:45,981] Trial 1 finished with value: 0.5739613913642438 and parameters: {'tree_method': 'hist', 'min_child_weight': 165, 'booster': 'dart', 'lambda': 1.3444878588290625e-08, 'alpha': 0.00012415269144787328, 'subsample': 0.7815391740980985, 'colsample_bynode': 0.9953887884481677, 'reg_lambda': 0.0062135183048939386, 'max_depth': 8, 'eta': 3.5825517390305215e-07, 'gamma': 0.00032202969734814314, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 4.3565117104846105e-05, 'skip_drop': 0.004694665649836412}. Best is trial 1 with value: 0.5739613913642438.


...

Use depth as feature
[0]	validation-rmse:1.02460
[1]	validation-rmse:0.88912
[2]	validation-rmse:0.82907
[3]	validation-rmse:0.80070
[4]	validation-rmse:0.77899
[5]	validation-rmse:0.76472
[6]	validation-rmse:0.74315
[7]	validation-rmse:0.73985
[8]	validation-rmse:0.73633
[9]	validation-rmse:0.72714


[I 2024-12-17 20:32:52,760] Trial 998 finished with value: 0.7520539204314283 and parameters: {'tree_method': 'hist', 'min_child_weight': 10, 'booster': 'gbtree', 'lambda': 0.06291336397367578, 'alpha': 1.6532982335341746e-05, 'subsample': 0.8460723267225377, 'colsample_bynode': 0.6233245435106933, 'reg_lambda': 13.329512303323789, 'max_depth': 12, 'eta': 0.22503876183276378, 'gamma': 0.0010026309646806288, 'grow_policy': 'depthwise'}. Best is trial 875 with value: 0.7939528240073528.


Use depth as feature
[0]	validation-rmse:1.05776
[1]	validation-rmse:0.92699
[2]	validation-rmse:0.86705
[3]	validation-rmse:0.84018
[4]	validation-rmse:0.82450
[5]	validation-rmse:0.81788
[6]	validation-rmse:0.80840
[7]	validation-rmse:0.80004
[8]	validation-rmse:0.79093
[9]	validation-rmse:0.78784


[I 2024-12-17 20:32:52,933] Trial 999 finished with value: 0.6803874768193319 and parameters: {'tree_method': 'hist', 'min_child_weight': 1, 'booster': 'gbtree', 'lambda': 0.0001571324420090598, 'alpha': 3.705022672044442e-05, 'subsample': 0.682312343843781, 'colsample_bynode': 0.4762089624020551, 'reg_lambda': 7.5326817081643656, 'max_depth': 11, 'eta': 0.1607254230242785, 'gamma': 2.7064602992943185e-06, 'grow_policy': 'depthwise'}. Best is trial 875 with value: 0.7939528240073528.


Number of finished trials:  1000
Best trial:
  Value: 0.7939528240073528
  Params: 
    tree_method: hist
    min_child_weight: 6
    booster: gbtree
    lambda: 8.245756366238077e-06
    alpha: 0.0013304944113653806
    subsample: 0.9574694449516479
    colsample_bynode: 0.5301122365172396
    reg_lambda: 11.178868632736467
    max_depth: 12
    eta: 0.11852260175023638
    gamma: 0.0004254341092206034
    grow_policy: depthwise


In [ ]:
    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


Number of finished trials:  1000
Best trial:
  Value: 0.795680889700473
  Params: 
    tree_method: approx
    min_child_weight: 1
    booster: dart
    lambda: 0.001396330409814792
    alpha: 0.949759668193326
    subsample: 0.9370354245159198
    colsample_bynode: 0.6649894630084585
    reg_lambda: 17.41021780297294
    max_depth: 11
    eta: 2.3464956481246766e-08
    gamma: 4.061922535832828e-07
    grow_policy: depthwise
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.00011689360891049891
    skip_drop: 3.7849279489120896e-07
